In [2]:
# This notebook will be used to run the model in a way that makes it easier to develop new components of the model
import nasem_dairy as nd
import pandas as pd
import math
from icecream import ic


def run_dev_model():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    # diet_info, animal_input, equation_selection = nd.read_csv_input('./input.csv')
    diet_info, animal_input, equation_selection = nd.read_csv_input('./input_drycow.csv')

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = diet_info['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    # Set feed inclusion percentages
    Fd_DMInp_Sum = diet_info['kg_user'].sum()  # Total intake, kg
    diet_info['Fd_DMInp'] = diet_info['kg_user'] / Fd_DMInp_Sum

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

########################################
# Step 2: DMI Equations
########################################
    # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = nd.NDF_precalculation(diet_info, feed_data)

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8: 
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'], 
            animal_input['Trg_MilkProd'], 
            animal_input['An_BW'], 
            animal_input['An_BCS'],
            animal_input['An_LactDay'], 
            animal_input['Trg_MilkFatp'], 
            animal_input['Trg_MilkTPp'], 
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers    
    elif equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'], 
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF, 
            animal_input['An_BW'], 
            animal_input['An_BW_mature'], 
            animal_input['An_PrePartWk'], 
            nd.coeff_dict)

    
    elif equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'], 
            animal_input['An_BW'], 
            animal_input['An_PrePartWk'], 
            animal_input['An_GestDay'], 
            animal_input['An_GestLength'], 
            Dt_NDF, 
            nd.coeff_dict)
        
    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: "+ str(equation_selection['DMIn_eqn']) )

    del(Dt_NDF)
########################################
# Step 3: Feed Based Calculations
########################################
    diet_info = nd.get_nutrient_intakes(diet_info, feed_data, animal_input['DMI'], equation_selection, nd.coeff_dict)

########################################
# Step 4: Micronutrient Calculations
########################################
    df_minerals, mineral_values = nd.mineral_intakes(animal_input['An_StatePhys'], feed_data, diet_info)

    df_vitamins = nd.vitamin_supply(feed_data, diet_info)
    
########################################
# Step 5: Microbial Protein Calculations
########################################
    Du_MiN_NRC2021_g, Rum_DigNDFIn, Rum_DigStIn = nd.calculate_Du_MiN_g(diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], diet_info.loc['Diet', 'Fd_St_kg/d'], diet_info.loc['Diet', 'Fd_CP_kg/d'], diet_info.loc['Diet', 'Fd_ADF_kg/d'], 
                                    diet_info.loc['Diet', 'Fd_ForWetIn'], diet_info.loc['Diet', 'Fd_RUPIn'], diet_info.loc['Diet', 'Fd_ForNDFIn'], diet_info.loc['Diet', 'Dt_RDPIn'], nd.coeff_dict)    
    
########################################
# Step 6: Amino Acid Calculations
########################################
    AA_values, Du_MiCP_g = nd.AA_calculations(Du_MiN_NRC2021_g, feed_data, diet_info, animal_input, nd.coeff_dict)

########################################
# Step 7: Other Calculations
########################################
# Intake calculations that require additional steps, need results from other calculations or values that need to be calculated for other functions

    GrUter_BWgain, GrUter_Wt = nd.calculate_GrUter_BWgain(animal_input['Fet_BWbrth'], animal_input['An_AgeDay'], animal_input['An_GestDay'], animal_input['An_GestLength'], animal_input['An_LactDay'], animal_input['An_Parity_rl'], nd.coeff_dict)

    # This function could be renamed as it is doing all the DE intake calculations
    An_DEIn, An_DENPNCPIn, An_DETPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, Fe_CP, Fe_CPend_g, Du_idMiCP_g = nd.calculate_An_DEIn(diet_info.loc['Diet', 'Fd_DigNDFIn_Base'], diet_info.loc['Diet', 'Fd_NDFIn'], 
                                                                                                                    diet_info.loc['Diet', 'Fd_DigStIn_Base'], diet_info.loc['Diet', 'Fd_St_kg/d'], 
                                                                                                                    diet_info.loc['Diet','Fd_DigrOMtIn'], diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_RUPIn'],
                                                                                                                    diet_info.loc['Diet', 'Fd_idRUPIn'], diet_info.loc['Diet', 'Fd_NPNCPIn'], diet_info.loc['Diet', 'Fd_DigFAIn'], 
                                                                                                                    Du_MiCP_g, animal_input['An_BW'], animal_input['DMI'], equation_selection['Monensin_eqn'], nd.coeff_dict)
    # Metabolizable Protein Intake
    An_MPIn, An_MPIn_g = nd.calculate_An_MPIn_g(diet_info.loc['Diet', 'Fd_idRUPIn'], Du_idMiCP_g, nd.coeff_dict)


    # Predicted milk protein
    Mlk_NP_g, An_DigNDF, An_DEInp = nd.calculate_Mlk_NP_g(AA_values, An_MPIn_g, An_DEIn, An_DETPIn, An_DENPNCPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, animal_input['An_BW'], animal_input['DMI'], animal_input['An_StatePhys'], nd.coeff_dict)

    # Gaseous energy loss
    An_GasEOut = nd.temp_calc_An_GasEOut(An_DigNDF, animal_input['An_StatePhys'], diet_info, animal_input['DMI'], equation_selection['Monensin_eqn'])

    # Net energy/Metabolizable energy
    An_NE, An_NE_In, An_MEIn, Frm_NPgain = nd.calculate_An_NE(diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_FAIn'], Mlk_NP_g, An_DEIn, An_DigNDF, Fe_CP, Fe_CPend_g, animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], 
                                     animal_input['Trg_FrmGain'], animal_input['Trg_RsrvGain'], GrUter_BWgain, An_GasEOut, nd.coeff_dict)

########################################
# Step 8: Requirement Calculations
########################################
    # Metabolizable Energy Requirements
    Trg_MEuse, An_MEmUse, An_MEgain, Gest_MEuse, Trg_Mlk_MEout, Trg_NEmilk_Milk, Frm_NEgain, Rsrv_NEgain = nd.calculate_ME_requirement(animal_input['An_BW'], animal_input['DMI'], animal_input['Trg_MilkProd'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                          animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'], animal_input['Trg_RsrvGain'], GrUter_BWgain, nd.coeff_dict)
    
    # Calculate some values for the heifer adjustment to MP requirement, this will be changed in the future and is placed here to avoid cluttering the calculate_MP_requirement function  
    An_DigTPaIn = nd.temp_calc_An_DigTPaIn(Fe_CP, diet_info)


    # Metabolizable Protein Requirements
    An_MPuse_g_Trg, An_MPm_g_Trg, Body_MPUse_g_Trg, Gest_MPUse_g_Trg, Mlk_MPUse_g_Trg, An_MPuse_kg_Trg = nd.calculate_MP_requirement(An_DEInp, An_DENPNCPIn, An_DigTPaIn, An_GasEOut, Frm_NPgain, diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                                                 animal_input['Trg_RsrvGain'], animal_input['Trg_MilkProd'], animal_input['Trg_MilkTPp'], GrUter_BWgain, animal_input['An_StatePhys'], nd.coeff_dict)    

########################################
# Step 9: Performance Calculations
########################################

    if animal_input['An_StatePhys'] == 'Lactating Cow':
        
        An_LactDay_MlkPred = nd.check_animal_lactation_day(animal_input['An_LactDay'])

        # Predicted milk fat
        Mlk_Fat_g = nd.calculate_Mlk_Fat_g(AA_values, diet_info.loc['Diet', 'Fd_FAIn'], diet_info.loc['Diet', 'Fd_DigC160In'], diet_info.loc['Diet', 'Fd_DigC183In'], animal_input['An_LactDay'], animal_input['DMI'])

        # Predicted milk yield
        Mlk_Prod_comp = nd.calculate_Mlk_Prod_comp(Mlk_NP_g, Mlk_Fat_g, An_DEIn, An_LactDay_MlkPred, animal_input['An_Parity_rl']) 
    
        # MP Allowable Milk
        Mlk_Prod_MPalow = nd.calculate_Mlk_Prod_MPalow(An_MPuse_g_Trg, Mlk_MPUse_g_Trg, An_MPIn, animal_input['Trg_MilkTPp'], nd.coeff_dict)

        # NE Allowable Milk
        Mlk_Prod_NEalow = nd.calculate_Mlk_Prod_NEalow(An_MEIn, An_MEgain, An_MEmUse, Gest_MEuse, Trg_NEmilk_Milk, nd.coeff_dict)
    else:
        An_LactDay_MlkPred = 0  # Default to 0 for non lactating animals
        Mlk_Fat_g = 0
        Mlk_Prod_comp = 0
        Mlk_Prod_MPalow = 0
        Mlk_Prod_NEalow = 0

########################################
# Step 10: Calculations Requiring Milk Production Values
########################################

    if animal_input['An_StatePhys'] == 'Lactating Cow':
        MlkNP_Milk = nd.temp_MlkNP_Milk(animal_input['An_StatePhys'], Mlk_NP_g, Mlk_Prod_comp, animal_input['Trg_MilkProd'])
    else:
        MlkNP_Milk = 0
    
    # Mineral Requirements
    mineral_requirements, mineral_balance, An_DCADmeq = nd.mineral_requirements(animal_input['An_StatePhys'], animal_input['An_Parity_rl'], animal_input['An_Breed'], animal_input['DMI'], animal_input['An_BW_mature'], animal_input['An_BW'], animal_input['Trg_FrmGain'], animal_input['Trg_RsrvGain'], animal_input['An_GestDay'], GrUter_Wt, Mlk_NP_g, animal_input['Trg_MilkProd'], animal_input['Trg_MilkTPp'],
                                                                mineral_values.loc['Ca', 'Abs_mineralIn'], MlkNP_Milk, mineral_values.loc['P', 'Abs_mineralIn'], mineral_values.loc['P', 'Dt_mineralIn'], mineral_values.loc['Mg', 'Abs_mineralIn'], mineral_values.loc['Na', 'Abs_mineralIn'], mineral_values.loc['Cl', 'Abs_mineralIn'], mineral_values.loc['K', 'Abs_mineralIn'],
                                                                mineral_values.loc['S', 'Dt_mineralIn'], mineral_values.loc['Co', 'Abs_mineralIn'], mineral_values.loc['Cu', 'Abs_mineralIn'], mineral_values.loc['I', 'Dt_mineralIn'], mineral_values.loc['Fe', 'Abs_mineralIn'], mineral_values.loc['Mn', 'Abs_mineralIn'], mineral_values.loc['Se', 'Dt_mineralIn'], 
                                                                mineral_values.loc['Zn', 'Abs_mineralIn'], mineral_values.loc['K', 'Dt_macro'], mineral_values.loc['Na', 'Dt_macro'], mineral_values.loc['Cl', 'Dt_macro'], mineral_values.loc['S', 'Dt_macro'])

    Mlk_Prod = nd.calculate_Mlk_Prod(animal_input['An_StatePhys'], 
                                  equation_selection['mProd_eqn'], 
                                  Mlk_Prod_comp, 
                                  Mlk_Prod_NEalow, 
                                  Mlk_Prod_MPalow, 
                                  animal_input['Trg_MilkProd']
                                  )

    MlkNE_Milk = nd.calculate_MlkNE_Milk(Mlk_Prod, 
                                      Mlk_Fat_g, 
                                      MlkNP_Milk, 
                                      animal_input['Trg_MilkLacp']
                                      )
    
    Mlk_MEout = nd.calculate_Mlk_MEout(Mlk_Prod,
                                    MlkNE_Milk, 
                                    nd.coeff_dict
                                    )

########################################
# Step 11: *Temporary* Add values of interest to list
########################################
    if animal_input['An_StatePhys'] == 'Lactating Cow':
        # Milk Fat %
        milk_fat = (Mlk_Fat_g / 1000) / Mlk_Prod_comp * 100
        # Milk Protein %
        milk_protein = (Mlk_NP_g / 1000) / Mlk_Prod_comp * 100
    else:
        milk_fat = None
        milk_protein = None

    # Metabolizable Protein Balance
    An_MPBal_g_Trg = An_MPIn_g - An_MPuse_g_Trg

    # Energy Balance
    An_MEuse = An_MEmUse + An_MEgain + Gest_MEuse + Mlk_MEout
    An_MEbal = An_MEIn - An_MEuse
    

    model_results = {
    'milk_fat': milk_fat,
    'milk_protein': milk_protein,
    'Mlk_Prod_comp': Mlk_Prod_comp,
    'Mlk_Prod_MPalow': Mlk_Prod_MPalow,
    'Mlk_Prod_NEalow': Mlk_Prod_NEalow,
    'Trg_MEuse': Trg_MEuse,
    'An_MPuse_g_Trg': An_MPuse_g_Trg,
    'An_MPBal_g_Trg': An_MPBal_g_Trg,
    'An_MEbal': An_MEbal        
    }

    ####################
    # Testing Statements
    ####################
    # print(An_DEIn)
    # print(An_MEIn)
    # print(An_MEbal)
    


    return diet_info, animal_input, feed_data, equation_selection, AA_values, model_results, df_minerals, mineral_values, df_vitamins, mineral_requirements, mineral_balance, An_DCADmeq

### RUN MODEL ###
diet_info, animal_input, feed_data, equation_selection, AA_values, model_results, df_minerals, mineral_values, df_vitamins, mineral_requirements, mineral_balance, An_DCADmeq = run_dev_model()


In [3]:
# Vitamin Requirements

def vitamin_requirement(Trg_MilkProd, An_Parity_rl, An_BW, Dt_VitAIn, Dt_VitDIn, Dt_VitEIn):
    # Vit A, IU/d
    if Trg_MilkProd > 35:
        An_VitA_req = 110 * An_BW + 1000 * (Trg_MilkProd - 35)
    else:
        An_VitA_req = 110 * An_BW
    
    An_VitA_bal = Dt_VitAIn - An_VitA_req

    # Vit D, IU/d
    if Trg_MilkProd > 0:
        An_VitD_req = 110 * An_BW + 1000 * (Trg_MilkProd - 35)
    else:
        An_VitD_req = 110 * An_BW
    
    An_VitD_bal = Dt_VitDIn - An_VitD_req

    # Vit E, IU/d
    if Trg_MilkProd == 0 and An_Parity_rl >= 1:
        An_VitE_req = 2.0 * An_BW
    else: 
        An_VitE_req = 0.8 * An_BW

    if An_StatePhys == 'Calf':
        An_VitE_req = 2.0 * An_BW
    elif An_GestDay >= 259 and An_Preg == 1:
        An_VitE_req = 3.0 * An_BW
    
    An_VitE_req = An_VitE_req - Dt_PastIn * 50  #50 IU provided per kg of pasture DM

    if An_VitE_req < 0:
        An_VitE_req = 0
    
    An_VitE_bal = Dt_VitEIn - An_VitE_req

    return An_VitA_req, An_VitD_req, An_VitE_req